*Alphafinder*

In this demo we build an Agent using the OpenAI assistants API, that can go fetch live data instead of relying on offline documentation. 


In [2]:
# first import what we need 

import json
import time
import os
import certifi
from urllib.request import urlopen
from openai import OpenAI
import gradio as gr

In [3]:
# Function to Fetch Stock Data
def fetch_stock_data(symbol):
    api_key = os.getenv("FINANCIAL_MODELING_PREP_API_KEY")
    url = f"https://financialmodelingprep.com/api/v3/quote/{symbol}?apikey={api_key}"
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)


In [4]:
# Function to fetch Earnings Transcripts
def fetch_earnings_transcript(symbol, year, quarter):
    api_key = os.getenv("FINANCIAL_MODELING_PREP_API_KEY") 
    url = f"https://financialmodelingprep.com/api/v3/earning_call_transcript/{symbol}?year={year}&quarter={quarter}&apikey={api_key}"
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    data_json = json.loads(data)
    for transcript in data_json:
        if len(transcript['content']) > 20000:
            transcript['content'] = transcript['content'][:20000]
    return data_json


In [5]:
# Add new function to tools
tools = [
    {
        "type": "function",
        "function": {
        "name": "fetch_stock_data",
        "description": "Fetches stock data for a given symbol",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol to query"
                }
            },
            "required": ["symbol"]
        }
    }
    },
    {
        "type": "function",
        "function": {
        "name": "fetch_earnings_transcript",
        "description": "Fetches earnings transcript for a given symbol, year, and quarter",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {
                    "type": "string",
                    "description": "The stock symbol to query"
                },
                "year": {
                    "type": "number",
                    "description": "The year of the earnings call"
                },
                "quarter": {
                    "type": "number",
                    "description": "The quarter of the earnings call"
                }
            },
            "required": ["symbol", "year", "quarter"]
        }
    }
    }
]


In [6]:
# Initialize OpenAI client with your API key
client = OpenAI(
    api_key= os.getenv("OPENAI_API_KEY"),
)

In [7]:
# Function to create an AI assistant
def create_assistant():
    assistant = client.beta.assistants.create(
        name="JPM DeepFin Demo",
        instructions= '''You are a highly intelligent, concise, and expert AI assistant that provides financial insights with a critical lens. You can fetch stock data and earnings transcripts to help users make informed decisions. Make sure you summarize the most important and relevant data as your {selection} and then provide linked logical {inferences} on top of that data. Make sure to format your outputs in a clear, concise manner, in  valid markdown. Take a deep breath and think step by step before you give your answer.''',
        model="gpt-4-turbo-preview",
        tools=tools
    )
    return assistant


In [8]:
# Function to create a thread
def create_thread():
    thread = client.beta.threads.create()
    return thread


In [9]:
# Function to send a message and run the thread
def send_and_run(content, thread, assistant):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )


In [10]:
# Function to get the response from a thread
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")


In [11]:
# Function to wait for a run to finish
def wait_on_run(run, thread, assistant):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    if run.status == "requires_action":
        tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "fetch_stock_data":
            task = fetch_stock_data(**arguments)
        elif tool_call.function.name == "fetch_earnings_transcript":
            task = fetch_earnings_transcript(**arguments)
        run = send_and_run(json.dumps(task), thread, assistant)
    return run


In [14]:
# Function for Gradio Interface. Fetches the stock data and transcript for the given ticker
def fetch_and_process_data(ticker_text, ticker_dropdown, data_type, year, quarter):
    ticker = ticker_text if ticker_text else ticker_dropdown
    assistant = create_assistant()
    thread = create_thread()

    if data_type == "Stock data":
        # Fetch stock data and send to the assistant
        stock_data = fetch_stock_data(ticker)
        run = send_and_run(json.dumps(stock_data), thread, assistant)
        run = wait_on_run(run, thread, assistant)
    elif data_type == "Earnings transcript":
        # Fetch earnings transcript and send to the assistant
        transcript = fetch_earnings_transcript(ticker, year, quarter)
        run = send_and_run(json.dumps(transcript), thread, assistant)
        run = wait_on_run(run, thread, assistant)

    responses = get_response(thread)
    response_text = ""
    for message in responses.data:
        if message.role == "assistant":
            response_text += str(message.content) + "\n\n"
    return response_text


In [15]:
tickers = ["AAPL", "GOOG", "AMZN", "META", "TSLA", "V", "MA", "JNJ", "KO", "NFLX", 
           "DIS", "WFC", "SBUX", "CMG", "PYPL", "EA", "SAM", "JWN"]


This is the cell that launches our basic, live agent app. Note that stock data is latest stock data (daily) whilst we need to specify year and quarter for earnings transcripts. Note that the earnings transcript analysis takes a lot longer to run than the stock price data.

In [17]:
interface = gr.Interface(
    fn=fetch_and_process_data,
    inputs=[
        gr.Textbox(placeholder="Enter ticker", label="Ticker (Text)"),
        gr.Dropdown(choices=tickers, label="Ticker (Dropdown)"),
        gr.Dropdown(choices=["Stock data", "Earnings transcript"], label="Data Type"),
        gr.Dropdown(choices=[2020, 2021, 2022, 2023, 2024], label="Year"),
        gr.Dropdown(choices=[1, 2, 3, 4], label="Quarter"),
    ],
    outputs=gr.Markdown(),
)

interface.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


/var/folders/cv/cxy_75z94pl1z5bm6xfj_khh0000gp/T/ipykernel_38419/2141677510.py:5: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())
